In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 1: Read in the file

In [2]:
file_id='test_data.csv'
df_read=pd.read_csv(file_id)
df_read.columns=df_read.columns.str.lower().values
df_read.head(4)

,timestamp,type,x,y,z
0,1380053672,r,-0.020386,0.020549,0.001251
1,1380064168,r,-0.017982,0.020502,0.003713
2,486416714040954,heading,-19.308655,-8.518478,-29.290878
3,1380058567,a,-0.006042,0.003784,-1.001617


In [3]:
#What are the sensor types in the data file?
types_found=pd.unique(df_read['type'])
types_found

array(['r', 'heading', 'a', 'm'], dtype=object)

In [4]:
# The sampling frequency:
f_s=200

# 2: Extrapolation/Upsampling function

In [5]:
imu_sensor_types=['a','m','r']
imu_found=set(types_found).intersection(set(imu_sensor_types))

df_imu=df_read.ix[(df_read['type']=='a')|(df_read['type']=='m')|(df_read['type']=='r'),:]
df_imu.reset_index(inplace=True,drop=True)
df_imu['time']=(df_imu.timestamp-df_imu.timestamp.min())/1e6


time_uniform=np.linspace(df_imu.timestamp.min(), df_imu.timestamp.max(), np.ceil(f_s*df_imu.time.max()))

from scipy.interpolate import interp1d
from scipy import array
# Extrapolation code from: http://stackoverflow.com/questions/2745329/how-to-make-scipy-interpolate-give-an-extrapolated-result-beyond-the-input-range

def extrap1d(interpolator):
    xs = interpolator.x
    ys = interpolator.y

    def pointwise(x):
        if x < xs[0]:
            return ys[0]+(x-xs[0])*(ys[1]-ys[0])/(xs[1]-xs[0])
        elif x > xs[-1]:
            return ys[-1]+(x-xs[-1])*(ys[-1]-ys[-2])/(xs[-1]-xs[-2])
        else:
            return interpolator(x)

    def ufunclike(xs):
        return array(map(pointwise, array(xs)))

    return ufunclike

def extrapolate_1d(time_in,values_in,time_uniform):
    f_x = extrap1d(interp1d(time_in,values_in))
    values_out =f_x(time_uniform)
    return values_out



C:\toolkits\anaconda2-4.1.0\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# 3: Extrapolate the data and convert the $n_{readings} \times 3$ dataframe into $n_{samples} \times \left( 3n_{sensors}+2\right)$ dataframe, where $n_{samples}=ceil \left( f_s . timespan (df_{read}) \right)$

In [6]:
df_imu_out=pd.DataFrame(data=time_uniform,columns=['timestamp'])
imu_srt=sorted(list(imu_found))
sens_present=''.join(map(str, imu_srt))
df_imu_out['sens_present']=sens_present
import itertools
spatial_axes=list(['x','y','z'])
imu_axes_comb=list(itertools.product(imu_found,spatial_axes))
imu_axes_comb
for sens_ax in imu_axes_comb:
    time_in=df_imu.ix[df_imu.type==sens_ax[0],'timestamp']
    values_in=df_imu.ix[df_imu.type==sens_ax[0],sens_ax[1]]
    df_imu_out[sens_ax[0]+sens_ax[1]]=extrapolate_1d(time_in,values_in,time_uniform)
    


# 4: The format of the output is ..

In [7]:
df_imu_out.head(4)

,timestamp,sens_present,ax,ay,az,rx,ry,rz,mx,my,mz
0,1.380054e+09,amr,-0.006073,0.004948,-1.004369,-0.020386,0.020549,0.001251,-29.296599,177.540058,-171.808550
1,1.380059e+09,amr,-0.006042,0.003758,-1.001556,-0.019240,0.020526,0.002425,-29.263641,177.241402,-171.776276
2,1.380064e+09,amr,-0.006011,0.002568,-0.998742,-0.018094,0.020504,0.003599,-29.230683,176.942746,-171.744002
3,1.380069e+09,amr,-0.005988,0.001576,-0.995782,-0.016415,0.019978,0.002634,-29.197725,176.644090,-171.711727


# 5: *Functionizing* the conversion

In [ ]:
from scipy.interpolate import interp1d
from scipy import array
import itertools

def extrap1d(interpolator):
    xs = interpolator.x
    ys = interpolator.y

    def pointwise(x):
        if x < xs[0]:
            return ys[0]+(x-xs[0])*(ys[1]-ys[0])/(xs[1]-xs[0])
        elif x > xs[-1]:
            return ys[-1]+(x-xs[-1])*(ys[-1]-ys[-2])/(xs[-1]-xs[-2])
        else:
            return interpolator(x)

    def ufunclike(xs):
        return array(map(pointwise, array(xs)))

    return ufunclike

def extrapolate_1d(time_in,values_in,time_uniform):
    f_x = extrap1d(interp1d(time_in,values_in))
    values_out =f_x(time_uniform)
    return values_out


def imu_df_gen(df_read, f_s):
    ''' Linearly extrapolate the data and convert the $n_{readings} \times 3$ dataframe into 
    $n_{samples} \times \left( 3n_{sensors}+2\right)$ dataframe, where 
    $n_{samples}=ceil \left( f_s . timespan (df_{read}) \right)$ '''

    df_read.columns=df_read.columns.str.lower().values
    types_found=pd.unique(df_read['type'])
    imu_sensor_types=['a','m','r']
    imu_found=set(types_found).intersection(set(imu_sensor_types))

    df_imu=df_read.ix[(df_read['type']=='a')|(df_read['type']=='m')|(df_read['type']=='r'),:]
    df_imu.reset_index(inplace=True,drop=True)
    df_imu['time']=(df_imu.timestamp-df_imu.timestamp.min())/1e6

    time_uniform=np.linspace(df_imu.timestamp.min(), df_imu.timestamp.max(), np.ceil(f_s*df_imu.time.max()))

    # Extrapolation code from: http://stackoverflow.com/questions/2745329/how-to-make-scipy-interpolate-give-an-extrapolated-result-beyond-the-input-range

    df_imu_out=pd.DataFrame(data=time_uniform,columns=['timestamp'])
    imu_srt=sorted(list(imu_found))
    sens_present=''.join(map(str, imu_srt))
    df_imu_out['sens_present']=sens_present

    spatial_axes=list(['x','y','z'])
    imu_axes_comb=list(itertools.product(imu_found,spatial_axes))
    imu_axes_comb
    for sens_ax in imu_axes_comb:
        time_in=df_imu.ix[df_imu.type==sens_ax[0],'timestamp']
        values_in=df_imu.ix[df_imu.type==sens_ax[0],sens_ax[1]]
        df_imu_out[sens_ax[0]+sens_ax[1]]=extrapolate_1d(time_in,values_in,time_uniform)

    return df_imu_out

In [12]:
df_test=pd.read_csv('test_data.csv')
df_out=imu_df_gen(df_test,200)

C:\toolkits\anaconda2-4.1.0\lib\site-packages\ipykernel\__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
#Deviations if any?
(df_out.ix[:,2:]-df_imu_out.ix[:,2:]).max()

ax    0.0
ay    0.0
az    0.0
rx    0.0
ry    0.0
rz    0.0
mx    0.0
my    0.0
mz    0.0
dtype: float64